<a href="https://colab.research.google.com/github/mralamdari/Computer-Vision-Projects/blob/main/YOLO_v5_Wheet_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import os
import cv2
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output

# Clone YOLOv5

Clone the YOLOv5 repository, and move all the yolo files to your current working directory

In [2]:
!git clone https://github.com/ultralytics/yolov5
!mv yolov5/* ./

Cloning into 'yolov5'...
remote: Enumerating objects: 15978, done.
remote: Counting objects: 100% (147/147), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 15978 (delta 89), reused 118 (delta 75), pack-reused 15831
Receiving objects: 100% (15978/15978), 14.61 MiB | 21.49 MiB/s, done.
Resolving deltas: 100% (10958/10958), done.


In [7]:
!pip install -r requirements.txt
clear_output()

In [ ]:
os.enviroment['KAGGLE_']
! kaggle competitions download global-wheat-detection